# Web Scraping con Selenium en El Tiempo

Este notebook muestra cómo automatizar la extracción de noticias desde el periódico El Tiempo utilizando Selenium.

**¿Qué es Selenium?**   
Selenium es una herramienta que nos permite automatizar navegadores web. Es como si escribiéramos un robot que abre una página, navega, hace clic y lee texto, igual que un humano.

**¿Cómo funciona el script?**
1. Abre la página de búsqueda.
2. Extrae todos los enlaces a noticias.
3. Visita cada enlace y extrae la información requerida.
4. Regresa a la página de búsqueda.
5. Hace clic en “Siguiente” y repite el proceso hasta que no haya más noticias.

**¿Qué se almacena?**  
Los datos extraídos se guardan en listas de Python, y finalmente se consolidan en un DataFrame que se exporta a Excel.

In [ ]:
#Instalar selenium
!pip install selenium

In [ ]:
# Importamos las librerías necesarias para el scraping con Selenium

# webdriver: permite controlar el navegador (abrir páginas, hacer clics, etc.)
from selenium import webdriver

# By: se usa para seleccionar elementos HTML mediante su clase, ID, nombre de etiqueta, etc.
from selenium.webdriver.common.by import By

# Options: nos permite configurar opciones del navegador (como abrir en modo headless)
from selenium.webdriver.chrome.options import Options

# WebDriverWait: permite esperar de forma explícita a que ciertos elementos aparezcan en la página
from selenium.webdriver.support.ui import WebDriverWait

# expected_conditions (EC): contiene condiciones que podemos esperar, como "elemento visible", "elemento clickeable", etc.
from selenium.webdriver.support import expected_conditions as EC

# time: permite hacer pausas con time.sleep(), útil para evitar que la página se cargue demasiado rápido
import time

**¿Qué es un WebDriver?**  
Un WebDriver es el "puente" entre Python y el navegador (Chrome, Firefox, etc.).  
Cuando usamos webdriver.Chrome(), estamos diciéndole a Python: "Abre una ventana de Chrome y déjame controlarla".

El WebDriver:  
· Abre el navegador  
· Carga páginas web  
· Encuentra elementos HTML (como botones, enlaces, textos)  
· Interactúa con la página (clics, escritura, scroll, etc.)  

In [ ]:
# Configuramos las opciones del navegador Chrome:
options = webdriver.ChromeOptions()  # Creamos un objeto de configuración para personalizar cómo se abre el navegador.
# NOTA: No activamos el modo "headless" para ver el navegador funcionando.

**¿Qué es un User-Agent?**  
Un User-Agent es una cadena de texto que le dice al sitio web quién eres.

Los servidores usan esto para:  
· Mostrar una versión diferente de la página según el navegador  
· Bloquear bots (como los que no tienen User-Agent o usan uno raro)  

Con Selenium, podemos simular un navegador real agregando un User-Agent manualmente. Esto puede ayudar a evitar bloqueos y obtener el contenido completo.

In [ ]:
# Aquí estamos configurando un User-Agent personalizado.
# Esto le dice al sitio web "soy un navegador real, no un bot".
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
# Usa este si trabajas en Windows: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36

In [ ]:
# Añadimos el User-Agent a las opciones del navegador:
options.add_argument(f"user-agent={user_agent}")

In [ ]:
# Lanzamos el navegador con las opciones configuradas:
driver = webdriver.Chrome(options=options)

In [ ]:
# Definimos la URL semilla desde donde comenzará el scraping. 
# Puedes cambiar esta URL si deseas buscar otras palabras clave o categorías en El Tiempo.
url_semilla = "https://www.eltiempo.com/buscar/?q=fiebre%20amarilla&sort_search=desc&categories_ids_or=58&multiFields=&articleTypes=especial_modular%2Cespecial-tipo-d%2Cdefault%2Copinion%2Cobituario%2Ccarta%2Cpodcast%2Ceditorial%2Cforo%2Ccaricatura%2Cinfografia&from=&until="

**¿Por qué usar "esperas" (waits)?**  
Cuando abrimos una página web, el contenido no siempre carga al instante, especialmente en sitios que usan JavaScript.

Si intentamos buscar elementos demasiado pronto, obtendremos errores.

Por eso usamos:  
· WebDriverWait: para esperar de forma inteligente.  
· expected_conditions: para decir "espera hasta que aparezca cierto elemento".  

In [ ]:
# Le indicamos al navegador que abra la URL semilla, que contiene los resultados de búsqueda.
driver.get(url_semilla)

# Creamos un objeto de espera explícita que le dice a Selenium que espere hasta 10 segundos
# para que los elementos necesarios aparezcan en la página antes de continuar.
wait = WebDriverWait(driver, 10)

**¿Qué son los selectores?**  
Cuando queremos encontrar un elemento en una página, usamos selectores.  

Estos se basan en atributos HTML como:  
· By.ID: por el atributo id  
· By.CLASS_NAME: por la clase CSS  
· By.TAG_NAME: por el nombre de la etiqueta (como div, a, etc.)  
· By.XPATH: usando expresiones XPath más avanzadas  
· By.CSS_SELECTOR: usando selectores CSS como en estilos  

**Ahora...**  
Una vez ubicados los elementos, usamos al final de la ruta:  
· click(): Esto simula el acto de hacer clic en el enlace identificado  
· text(): Extrae la información en formato texto

In [ ]:
# Localizamos el primer enlace de noticia usando su clase CSS y simulamos un clic para acceder a la noticia.
driver.find_element(By.CLASS_NAME, "c-article__title").click()

In [ ]:
# Localizamos el elemento que contiene el título completo de la noticia una vez que estamos dentro del artículo.
titulo_noticia = driver.find_element(By.CLASS_NAME, "c-articulo__titulo").text
# Mostramos el título en la consola
print(titulo_noticia)

In [ ]:
# Ahora buscamos el subtítulo usando la clase CSS del contenedor de subtítulos.
subtitulo_noticia = driver.find_element(By.CLASS_NAME, "c-lead__titulo").text
print(subtitulo_noticia)

In [ ]:
# Extraemos el cuerpo de la noticia, que también está identificado por una clase CSS.
contenido_noticia = driver.find_element(By.CLASS_NAME, "c-cuerpo").text
print(contenido_noticia)

In [ ]:
# Extraemos la fecha de publicación de la noticia.
# Usamos el nombre de la etiqueta <time>, ya que normalmente es único en la página y contiene la fecha.
fecha_noticia = driver.find_element(By.TAG_NAME, "time").text
print(fecha_noticia)

Logramos extraer la información de la primera noticia de nuestra búsqueda. Sin embargo, nuestro objetivo es automatizar todo el proceso para que el navegador recorra cada enlace de noticia en la página de resultados, acceda a cada uno, y extraiga el título, subtítulo, contenido y fecha. Al finalizar con todos los enlaces de la página actual, el navegador debe hacer clic en el botón "Siguiente" para continuar con la siguiente página de resultados y repetir el mismo procedimiento.

Para almacenar la información que extraeremos, crearemos listas vacías donde guardaremos los datos recolectados de cada noticia: el enlace (URL), el título, el subtítulo, el contenido y la fecha de publicación. Estas listas se llenarán automáticamente dentro de un bucle que recorrerá todas las noticias de cada página. A medida que el navegador visite cada noticia, iremos extrayendo los datos y agregándolos a las listas correspondientes.

In [ ]:
# Creamos listas vacías para almacenar la información extraída de cada noticia
urls = []         # Aquí se guardarán los enlaces a cada noticia
titulos = []      # Aquí se almacenarán los títulos de las noticias
subtitulos = []   # Aquí se almacenarán los subtítulos (si existen)
contenidos = []   # Aquí se guardará el cuerpo completo del texto de cada noticia
fechas = []       # Aquí se guardarán las fechas de publicación

**¡Recordatorio!**  
Una lista en Python es una estructura que permite almacenar múltiples elementos en un solo objeto. Se define con corchetes [] y puede crecer dinámicamente. En este caso, usaremos listas para ir guardando la información de cada noticia que vayamos extrayendo durante el proceso de scraping.

In [ ]:
# Iniciamos un bucle que continuará hasta que ya no haya más páginas
while True:
    # Esperamos que se carguen todos los enlaces a noticias en la página
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//h3/a[@href]")))
    # Obtenemos todos los elementos <a> que representan los enlaces a las noticias
    article_links = driver.find_elements(By.XPATH, "//h3/a[@href]")
    
    # Creamos una lista con los href únicos extraídos de cada enlace
    current_page_urls = [link.get_attribute("href") for link in article_links if link.get_attribute("href")]
    
    # Recorremos cada URL de noticia encontrada en la página actual
    for url in current_page_urls:
        try:
            # Abrimos la URL de la noticia en una nueva pestaña del navegador
            driver.execute_script("window.open(arguments[0]);", url)
            # Cambiamos el control a la nueva pestaña abierta
            driver.switch_to.window(driver.window_handles[1])
            # Esperamos que se cargue el título de la noticia
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "c-articulo__titulo")))
            
            # Extraemos el título de la noticia
            title = driver.find_element(By.CLASS_NAME, "c-articulo__titulo").text
            try:
                # Extraemos el subtítulo de la noticia si existe
                subtitle = driver.find_element(By.CLASS_NAME, "c-lead__titulo").text
            except:
                subtitle = ""
            try:
                # Extraemos todos los párrafos del cuerpo de la noticia
                body_paragraphs = driver.find_elements(By.CLASS_NAME, "c-cuerpo")
                # Unimos los párrafos extraídos en un solo texto de contenido
                content = "\n".join(p.text for p in body_paragraphs)
            except:
                content = ""
            try:
                # Extraemos la fecha de publicación de la noticia
                date = driver.find_element(By.TAG_NAME, "time").text
            except:
                date = ""
            
            # Guardamos la URL y los datos extraídos en sus respectivas listas
            urls.append(url)
            titulos.append(title)
            subtitulos.append(subtitle)
            contenidos.append(content)
            fechas.append(date)

        # Si ocurre un error al procesar la noticia, lo mostramos por consola
        except Exception as e:
            print(f"Error procesando {url}: {e}")
        finally:
            # Cerramos la pestaña actual de la noticia
            driver.close()
            # Volvemos a la pestaña de resultados de búsqueda
            driver.switch_to.window(driver.window_handles[0])

    # Pasamos a la siguiente página
    try:
        # Buscamos el botón de 'Siguiente' para avanzar de página
        next_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "next")))
        # Hacemos clic en el botón de 'Siguiente'
        next_button.click()
        # Esperamos un momento para asegurar que la página siguiente cargue correctamente
        time.sleep(2)
    except:
        print("No hay más páginas.")
        # Si no hay más botón de 'Siguiente', terminamos el bucle
        break

# Cerramos completamente el navegador al terminar
driver.quit()

In [ ]:
# Mostramos la lista completa de URLs extraídas
print(urls)

In [ ]:
# Mostramos la lista completa de títulos extraídos
print(titulos)

In [ ]:
# Mostramos la lista completa de subtítulos extraídos
print(subtitulos)

In [ ]:
# Mostramos la lista completa de contenidos extraídos
print(contenidos)

In [ ]:
# Mostramos la lista completa de fechas extraídas
print(fechas)

In [ ]:
# Imprime la cantidad de elementos extraídos en cada lista.
# Esto nos ayuda a verificar que todas las listas tengan la misma longitud.
print(len(urls))         # Número de URLs extraídas
print(len(titulos))      # Número de títulos extraídos
print(len(subtitulos))   # Número de subtítulos extraídos
print(len(contenidos))   # Número de contenidos extraídos
print(len(fechas))       # Número de fechas extraídas

**¡Nota!**  
Para poder crear un DataFrame, es importante que todas las listas tengan la misma longitud. Cada fila del DataFrame representa una noticia, por lo que si alguna lista tiene más o menos elementos, pandas no podrá alinear los datos correctamente y generará un error.

In [ ]:
#Instalar pandas (en caso de que no la tengamos)
!pip install selenium

In [ ]:
# Importamos la biblioteca pandas, que nos permite manipular datos en forma de tablas (DataFrames).
import pandas as pd

**¡Recordatorio!**  
Un diccionario en Python es una estructura de datos que almacena pares de clave y valor. Las claves (por ejemplo, "Título", "Fecha") actúan como etiquetas, y cada una apunta a un valor (en este caso, una lista de datos extraídos). Se define usando llaves {}. En nuestro caso, usaremos un diccionario para agrupar todas las listas de noticias extraídas y luego convertirlo en un DataFrame.

In [ ]:
# Creamos un diccionario donde agrupamos toda la información extraída por campos: URL, título, subtítulo, contenido y fecha.
noticias_eltiempo_migracion = {
    'URL': urls,
    'Título': titulos,
    'Subtítulo': subtitulos,
    'Contenido': contenidos,
    'Fecha': fechas
}

# Convertimos el diccionario en un DataFrame de pandas, que es como una hoja de cálculo en memoria.
df = pd.DataFrame(noticias_eltiempo_migracion)
df # Mostramos el DataFrame como salida en Jupyter.

In [ ]:
# Exportamos el DataFrame a un archivo Excel. Usamos index=False para no guardar los índices de las filas.
df.to_excel('noticias_eltiempo_fiebreamarilla.xlsx', index=False)